In [7]:
import numpy as np
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, SubsetRandomSampler
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.49139969, 0.48215842, 0.44653093], [0.20220212, 0.19931542, 0.20086347])
])
training_data = CIFAR10('cifar/train', train=True, download=True, transform=transform)
val_data = CIFAR10('cifar/train', train=True, download=True, transform=transform)
test_data = CIFAR10('cifar/test', train=False, download=True, transform=transform)
N = len(training_data)

loader_train = DataLoader(training_data, batch_size=128, shuffle=False, sampler=SubsetRandomSampler(range(0,N-5000)))
loader_val = DataLoader(val_data, batch_size=128, shuffle=False, sampler=SubsetRandomSampler(range(N-5000,N)))
loader_test = DataLoader(test_data, 128)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [3]:
class Resnet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1, 1)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()
        self.block1 = self._create_block(n, 16, 16, 1)
        self.block2 = self._create_block(n, 16, 32, 2)
        self.block3 = self._create_block(n, 32, 64, 2)
        self.pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, 10)
    
    def _create_block(self, n, num_input, num_output, stride):
        sub_blocks = [Resblock(num_input, num_output, stride)]
        for l in range(n-1):
            sub_blocks.append(Resblock(num_output, num_output, 1))
        return nn.Sequential(*sub_blocks)
        
    def forward(self,x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.pool(out)
        out = out.view(out.shape[0], -1)
        out = self.fc(out)
        return out

In [4]:
class Resblock(nn.Module):
    def __init__(self, num_input, num_output, stride):
        super().__init__()
        self.conv1 = nn.Conv2d(num_input, num_output, 3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(num_output)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(num_output, num_output, 3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_output)
        self.relu2 = nn.ReLU()
        self.identity = IdentityMapping(stride, num_input, num_output)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu2(out)
        out += self.identity(x)
        return out

In [5]:
#Option A from paper, allows to change size of input image, if necessary (size/2 when stride=2)
#This helps to match sizes of input resblock and output of resblock
class IdentityMapping(nn.Module):
    def __init__(self, stride, num_input, num_output):
        super().__init__()
        self.maxpool = nn.MaxPool2d(kernel_size=1, stride=stride)
        self.extra_filters = num_output - num_input
        
    def forward(self, x):
        out = F.pad(x, (0, 0, 0, 0, 0, self.extra_filters))
        out = self.maxpool(out)
        return out

In [23]:
def get_accuracy(model, loader):
    acc = 0.0
    counter = 0
    for idx, (data, labels) in enumerate(loader):
        with torch.no_grad():
            x = data.to(device)
            y = labels.to(device)
            scores = model(x)
            guesses = scores.argmax(dim=1) == y
            acc += guesses.sum().float()/len(guesses)
            counter += 1
    return float(acc / float(counter))

In [26]:
model = Resnet(9)
model.to(device)
epochs=10
history = {
    'train_acc': [],
    'val_acc': [],
    'loss': []
}
for e in range(epochs):
    for idx, (data, labels) in enumerate(loader_train):
        data = data.to(device)
        labels = labels.to(device)
        scores = model(data)
        optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
        loss = F.cross_entropy(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if idx % 100:
            guesses = scores.argmax(dim=1)==labels
            train_acc = float(guesses.sum().float()/len(guesses))
            val_acc = get_accuracy(model, loader_val)
            history['train_acc'].append(acc)
            history['val_acc'].append(val_acc)
            history['loss'].append(float(loss))
            print('Epoch: ', e, 'iteration:', idx * 100)
            print('loss:', float(loss))
            print('train acc:', train_acc)
            print('validation acc:', val_acc)
            print('----------------------------------------------------')
        if idx % 1000 and idx != 0:
            torch.save(model.state_dict(), './resnet_trained.pt')

Epoch:  0 iteration: 100
loss: 8.735483169555664
train acc: 0.0859375
validation acc: 0.09804687649011612
----------------------------------------------------
Epoch:  0 iteration: 200
loss: 8.14089298248291
train acc: 0.09375
validation acc: 0.10000000149011612
----------------------------------------------------
Epoch:  0 iteration: 300
loss: 7.392881870269775
train acc: 0.0859375
validation acc: 0.09550781548023224
----------------------------------------------------
Epoch:  0 iteration: 400
loss: 10.641711235046387
train acc: 0.1015625
validation acc: 0.09648437798023224
----------------------------------------------------
Epoch:  0 iteration: 500
loss: 10.16873836517334
train acc: 0.078125
validation acc: 0.13515625894069672
----------------------------------------------------
Epoch:  0 iteration: 600
loss: 8.32387638092041
train acc: 0.09375
validation acc: 0.11582031100988388
----------------------------------------------------
Epoch:  0 iteration: 700
loss: 6.603109836578369
tra

----------------------------------------------------
Epoch:  0 iteration: 5300
loss: 2.4799201488494873
train acc: 0.1015625
validation acc: 0.21445313096046448
----------------------------------------------------
Epoch:  0 iteration: 5400
loss: 2.1845333576202393
train acc: 0.1640625
validation acc: 0.19160155951976776
----------------------------------------------------
Epoch:  0 iteration: 5500
loss: 2.3290700912475586
train acc: 0.171875
validation acc: 0.16523437201976776
----------------------------------------------------
Epoch:  0 iteration: 5600
loss: 2.3541831970214844
train acc: 0.125
validation acc: 0.17558594048023224
----------------------------------------------------
Epoch:  0 iteration: 5700
loss: 2.1860861778259277
train acc: 0.1484375
validation acc: 0.18691407144069672
----------------------------------------------------
Epoch:  0 iteration: 5800
loss: 2.2837207317352295
train acc: 0.1484375
validation acc: 0.2197265625
----------------------------------------------

validation acc: 0.24394531548023224
----------------------------------------------------
Epoch:  0 iteration: 10600
loss: 2.040327548980713
train acc: 0.234375
validation acc: 0.1904296875
----------------------------------------------------
Epoch:  0 iteration: 10700
loss: 2.1984877586364746
train acc: 0.1640625
validation acc: 0.23749999701976776
----------------------------------------------------
Epoch:  0 iteration: 10800
loss: 1.9097588062286377
train acc: 0.2734375
validation acc: 0.22734375298023224
----------------------------------------------------
Epoch:  0 iteration: 10900
loss: 2.0201940536499023
train acc: 0.2578125
validation acc: 0.22226563096046448
----------------------------------------------------
Epoch:  0 iteration: 11000
loss: 1.9537979364395142
train acc: 0.2421875
validation acc: 0.22500000894069672
----------------------------------------------------
Epoch:  0 iteration: 11100
loss: 2.183785915374756
train acc: 0.1953125
validation acc: 0.19570313394069672
--

----------------------------------------------------
Epoch:  0 iteration: 15700
loss: 1.9357508420944214
train acc: 0.2578125
validation acc: 0.2894531190395355
----------------------------------------------------
Epoch:  0 iteration: 15800
loss: 1.914411187171936
train acc: 0.3125
validation acc: 0.2562499940395355
----------------------------------------------------
Epoch:  0 iteration: 15900
loss: 1.8985590934753418
train acc: 0.2265625
validation acc: 0.2724609375
----------------------------------------------------
Epoch:  0 iteration: 16000
loss: 1.924173355102539
train acc: 0.2421875
validation acc: 0.28496095538139343
----------------------------------------------------
Epoch:  0 iteration: 16100
loss: 1.9784605503082275
train acc: 0.171875
validation acc: 0.2529296875
----------------------------------------------------
Epoch:  0 iteration: 16200
loss: 1.884498953819275
train acc: 0.2890625
validation acc: 0.22636719048023224
---------------------------------------------------

loss: 1.8420300483703613
train acc: 0.28125
validation acc: 0.296875
----------------------------------------------------
Epoch:  0 iteration: 21000
loss: 1.8814854621887207
train acc: 0.2421875
validation acc: 0.23203125596046448
----------------------------------------------------
Epoch:  0 iteration: 21100
loss: 1.8398165702819824
train acc: 0.2109375
validation acc: 0.2919921875
----------------------------------------------------
Epoch:  0 iteration: 21200
loss: 1.9444494247436523
train acc: 0.2265625
validation acc: 0.3041015565395355
----------------------------------------------------
Epoch:  0 iteration: 21300
loss: 1.8490453958511353
train acc: 0.3203125
validation acc: 0.3177734315395355
----------------------------------------------------
Epoch:  0 iteration: 21400
loss: 1.7966312170028687
train acc: 0.3125
validation acc: 0.28984376788139343
----------------------------------------------------
Epoch:  0 iteration: 21500
loss: 1.871015191078186
train acc: 0.328125
validatio

...

train acc: 0.78125
validation acc: 0.748242199420929
----------------------------------------------------
Epoch:  6 iteration: 24400
loss: 0.655559778213501
train acc: 0.765625
validation acc: 0.7564453482627869
----------------------------------------------------
Epoch:  6 iteration: 24500
loss: 0.6653205752372742
train acc: 0.7578125
validation acc: 0.7601562738418579
----------------------------------------------------
Epoch:  6 iteration: 24600
loss: 0.6513307094573975
train acc: 0.734375
validation acc: 0.7398437857627869
----------------------------------------------------
Epoch:  6 iteration: 24700
loss: 0.67554771900177
train acc: 0.75
validation acc: 0.759765625
----------------------------------------------------
Epoch:  6 iteration: 24800
loss: 0.5924407839775085
train acc: 0.7890625
validation acc: 0.766796886920929
----------------------------------------------------
Epoch:  6 iteration: 24900
loss: 0.6668956875801086
train acc: 0.7734375
validation acc: 0.7525390982627869

Epoch:  6 iteration: 29500
loss: 0.7619149684906006
train acc: 0.7109375
validation acc: 0.776171863079071
----------------------------------------------------
Epoch:  6 iteration: 29600
loss: 0.5970195531845093
train acc: 0.8046875
validation acc: 0.771289050579071
----------------------------------------------------
Epoch:  6 iteration: 29700
loss: 0.5723423957824707
train acc: 0.8046875
validation acc: 0.7666015625
----------------------------------------------------
Epoch:  6 iteration: 29800
loss: 0.408537894487381
train acc: 0.8515625
validation acc: 0.7789062857627869
----------------------------------------------------
Epoch:  6 iteration: 29900
loss: 0.5450780391693115
train acc: 0.84375
validation acc: 0.7730469107627869
----------------------------------------------------
Epoch:  6 iteration: 30100
loss: 0.6612210273742676
train acc: 0.7421875
validation acc: 0.7626953125
----------------------------------------------------
Epoch:  6 iteration: 30200
loss: 0.4726853370666504

----------------------------------------------------
Epoch:  6 iteration: 34800
loss: 0.4853999614715576
train acc: 0.84375
validation acc: 0.751171886920929
----------------------------------------------------
Epoch:  6 iteration: 34900
loss: 0.601041853427887
train acc: 0.8046875
validation acc: 0.7583984732627869
----------------------------------------------------
Epoch:  6 iteration: 35000
loss: 0.7453550696372986
train acc: 0.75
validation acc: 0.763867199420929
----------------------------------------------------
Epoch:  6 iteration: 35100
loss: 0.34913164377212524
train acc: 0.8611111044883728
validation acc: 0.7484375238418579
----------------------------------------------------
Epoch:  7 iteration: 100
loss: 0.44652169942855835
train acc: 0.796875
validation acc: 0.7660156488418579
----------------------------------------------------
Epoch:  7 iteration: 200
loss: 0.6113782525062561
train acc: 0.8046875
validation acc: 0.767382800579071
---------------------------------------

validation acc: 0.7748047113418579
----------------------------------------------------
Epoch:  7 iteration: 4900
loss: 0.4241686165332794
train acc: 0.8828125
validation acc: 0.7708984613418579
----------------------------------------------------
Epoch:  7 iteration: 5000
loss: 0.5919333100318909
train acc: 0.8125
validation acc: 0.76171875
----------------------------------------------------
Epoch:  7 iteration: 5100
loss: 0.5831888914108276
train acc: 0.7890625
validation acc: 0.7582031488418579
----------------------------------------------------
Epoch:  7 iteration: 5200
loss: 0.6842286586761475
train acc: 0.78125
validation acc: 0.7613281607627869
----------------------------------------------------
Epoch:  7 iteration: 5300
loss: 0.46239006519317627
train acc: 0.8359375
validation acc: 0.7720703482627869
----------------------------------------------------


KeyboardInterrupt: 